retrogaming-qa-bot

### 💻 **<i>Notebook 05:</i> Agent Evaluation**

**Since Colab sessions have a hard time maintaining variables from one session to another, this notebook reconstructs aspects from the previous ones to run safely**

In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s e

**For running on Google Colab only, run the code cell below:**

In [ ]:
import os
from dotenv import load_dotenv
try:
    # This import ONLY works on the Colab web interface
    from google.colab import userdata

    print("Loading API keys from Colab Secrets...")

    # --- Key Retrieval and Injection into os.environ ---
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')
    os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
    os.environ['LANGCHAIN_PROJECT'] = "retrogaming-qa-bot"

    # --- Verification ---
    if not os.environ.get('OPENAI_API_KEY'):
        raise ValueError("OPENAI_API_KEY was not found in Colab Secrets.")

    print("✅ All API keys were successfully loaded from Colab Secrets.")

except ImportError:
    # Fallback for local environments (local VS Code)
    print("Loading API keys from local .env file (VS Code/Local Mode).")
    load_dotenv()

# --- Common Configurations ---
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

# Here, the rest of the notebook variables are defined (llm, embeddings, etc.)
# ... (should continue defining LLM/RAG configuration variables)

A carregar chaves API do Colab Secrets...
✅ Todas as chaves API foram carregadas com sucesso a partir do Colab Secrets.


##### **00 - Imports, Dependencies and Environment**

In [2]:
# ================================================================
# 00 — DEPENDENCIES & ENVIRONMENT
# ================================================================

!pip install --quiet langsmith openevals

import os
import uuid
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv

from langsmith import Client
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT

# Load environment variables
try:
    # Este import SÓ funciona na interface web do Colab
    from google.colab import userdata

    print("A carregar chaves API do Colab Secrets...")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    os.environ["GOOGLE_CSE_ID"] = userdata.get("GOOGLE_CSE_ID")
    os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGCHAIN_API_KEY") # Atenção: 'LANGCHAIN_API_KEY' é usado nos Notebooks anteriores
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY") # Certifique-se que ambas existem
    print("✅ Chaves API carregadas do Colab Secrets.")

except ImportError:
    # Fallback para ambientes locais (VS Code local)
    print("A carregar chaves API do ficheiro .env local (Modo VS Code/Local).")
    load_dotenv()
    if not os.getenv("LANGCHAIN_API_KEY"):
         raise ValueError("LANGCHAIN_API_KEY não foi encontrada. Avaliação com LangSmith desativada.")

# Ensure non-empty vars for LangSmith
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "retrogaming-qa-bot"

print("Environment ready for evaluation.")

A carregar chaves API do Colab Secrets...
✅ Chaves API carregadas do Colab Secrets.
Environment ready for evaluation.


In [4]:
!pip install langchain_openai langchain_community langchain_core langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opentelemetry-exporter-gcp-logging 1.11.0a0 requires opentelemetry-sdk<1.39.0,>=1.35.0, but you have opentelemetry-sdk 1.39.0 which is incompatible.
google-adk 1.19.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.0 whic

##### **01 - Load Agent Executor**

In [3]:
# ================================================================
# 01 — LOAD AGENT EXECUTOR (Reconstruction from Notebook 03)
# ================================================================
import chromadb
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.tools import Tool
from langchain_classic.agents import initialize_agent, AgentType
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_community.tools import GoogleSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper


# --- Configuration (from Notebooks 01-04) ---
CHROMA_PATH = "./chroma_db_retrogaming"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"

llm = ChatOpenAI(model=LLM_MODEL, temperature=0.0)

# --- 1. Memory Definition ---
conversational_memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

# --- 2. RAG Tool Setup ---
print("\nReconstructing Retriever and RAG Tool...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
# NOTA: Assumimos que o ChromaDB foi persistido no Notebook 01
vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

def get_retrogaming_docs(query: str) -> str:
    """Fetches documents from the RAG knowledge base."""
    docs = retriever.invoke(query)
    formatted_docs = []
    for doc in docs:
        source_type = doc.metadata.get('source_type', 'unknown')
        formatted_docs.append(f"[{source_type.upper()}] {doc.page_content}")
    return "\n\n".join(formatted_docs)

retrogaming_qa_tool = Tool(
    name="retrogaming_knowledge_base",
    func=get_retrogaming_docs,
    description="Tool to search and retrieve specific information from the internal knowledge base (YouTube transcripts or Web tutorials) about emulators, settings, and consoles. Use this FIRST."
)

# --- 3. Google Search Tool Setup ---
search_wrapper = GoogleSearchAPIWrapper()
google_search_tool = GoogleSearchRun(
    name="google_search",
    api_wrapper=search_wrapper,
    description="Use this tool to search the internet for current or highly specific information when the internal knowledge base does not have the answer. Useful for new game specs or up-to-date emulator news."
)

tools = [retrogaming_qa_tool, google_search_tool]

# --- 4. System Prompt (from Notebook 03) ---
SYSTEM_TEMPLATE = (
    "You are a comprehensive expert retrogaming assistant and tuning specialist. Your expertise covers "
    "tuning old video games from various platforms (including PC) for better resolution and modern "
    "compatibility on PC, Mac, or the original native console hardware. "

    "***INITIAL CONVERSATION GUIDANCE:***"

    "1. **PC/Mac Path Flow:** If the user chooses PC or Mac:"
    "   - **Gather Specs:** IMMEDIATELY ask for the user's current **computer specs** (CPU/GPU/RAM) and their **monitor's native resolution**. **CRITICAL RULE:** If the user states they do not know how to find their computer specs, you MUST provide clear, step-by-step answers on how to find this information on their operating system (Windows or macOS)."
    "   - **Advice:** Once specs are known, use your tools to provide information on the available **emulators** (like PCSX2, Dolphin, etc.) and **tuning specs** tailored to their hardware for the specific game (e.g., Jak 2 Renegade)."

    "2. **Native Console Path Flow:** If the user chooses the native console:"
    "   - **Advice:** Use your tools to reply with the best options to output that system to its best possible resolution, naming **external third-party devices** like HDMI adapters or image/signal converters."

    "***HYBRID CONSTRAINT (RAG & SEARCH PRIORITY):***"
    "3. **PRIORITY:** You **MUST** attempt to use the tool **'retrogaming_knowledge_base'** first."
    "4. **FALLBACK:** If the required information is **NOT** found in the RAG context, you **MUST** use the tool **'google_search'** to find a reliable external source before formulating the final answer."
    "5. **TRANSPARENCY:** If the final answer is based on the Google search, briefly mention that the information was retrieved from an updated search."

    "\n\nChat History: {chat_history}"
    "\n\nUser Question: {input}"
)

# --- 5. Initialize Agent Executor ---
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True, # Mantemos True para rastreamento (tracing) no LangSmith
    memory=conversational_memory,
    agent_kwargs={"system_message": SYSTEM_TEMPLATE}
)

print("✅ Hybrid Agent Executor reconstructed and ready for evaluation.")

/tmp/ipython-input-1028515783.py:22: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(
/tmp/ipython-input-1028515783.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)



Reconstructing Retriever and RAG Tool...
✅ Hybrid Agent Executor reconstructed and ready for evaluation.


/tmp/ipython-input-1028515783.py:51: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchAPIWrapper``.
  search_wrapper = GoogleSearchAPIWrapper()
/tmp/ipython-input-1028515783.py:52: LangChainDeprecationWarning: The class `GoogleSearchRun` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchRun``.
  google_search_tool = GoogleSearchRun(
/tmp/ipython-input-1028515783.py:85: LangChainDeprecationWarning: LangChain agents will continue to be

##### **02 - Define Test Data anda Evaluators**

In [4]:
# ================================================================
# 02 — DEFINE TEST DATA AND EVALUATORS
# ================================================================

TEST_QUESTIONS = [
    "What setting should I check in a classic game to fix a stretched resolution issue?",
    "I choose PC/Mac, but I don't know my computer specs.",
    "What are the latest best PCSX2 settings for 'Jak 2 Renegade' in 2025?"
]

GROUND_TRUTHS = [
    "You should look for widescreen patches or ensure native widescreen/aspect ratio settings.",
    "The agent should provide steps to find CPU/GPU/RAM using operating system specific methods (e.g., DxDiag on Windows or About This Mac on macOS) before proceeding.",
    "The agent should use Google Search to retrieve modern PCSX2 settings for the year 2025, as this is dynamic information likely outside the RAG knowledge base."
]


# 1) LLM-as-a-Judge for correctness
correctness_judge = create_llm_as_judge(
    prompt=CORRECTNESS_PROMPT,
    feedback_key="correctness",
    model="openai:gpt-4o"
)

# 2) Relevance evaluator — lexical question/answer overlap
def relevance_evaluator(inputs, outputs, reference_outputs):
    from difflib import SequenceMatcher
    q = inputs["question"].lower()
    a = outputs["prediction"].lower()

    # Calculate Jaccard similarity between question words and answer words
    qw = set(q.split())
    aw = set(a.split())

    # Avoid division by zero
    if not qw:
        overlap = 0.0
    else:
        # A simple measure of how many question words appear in the answer (may not be the best)
        # Using a safer, more standard metric: Jaccard on unigrams is a reasonable approximation
        intersection = len(qw.intersection(aw))
        union = len(qw.union(aw))
        overlap = intersection / union if union else 0.0

    return {"key": "relevance", "score": overlap}

# 3) Semantic similarity evaluator — answer vs ground truth (using SequenceMatcher for lexical sim)
def similarity_evaluator(inputs, outputs, reference_outputs):
    from difflib import SequenceMatcher
    pred = outputs["prediction"]
    gt = reference_outputs["ground_truth"]

    # Ratio of matching blocks, a simple lexical similarity score
    sim = SequenceMatcher(None, pred.lower(), gt.lower()).ratio()

    return {"key": "similarity", "score": sim}

print("✅ Test data and evaluation functions defined.")

✅ Test data and evaluation functions defined.


##### **03 - Evaluation Loop**

In [5]:
# ================================================================
# 03 — EVALUATION LOOP (Uses your Hybrid Agent Executor)
# ================================================================

print("\nStarting evaluation loop...")

results = []

for i, (q, gt) in enumerate(zip(TEST_QUESTIONS, GROUND_TRUTHS)):
    print(f"\n--- Running Test Case {i+1}/{len(TEST_QUESTIONS)} ---")
    print(f"Question: {q}")

    # ---- CALL YOUR ACTUAL AGENT ----
    # NOTA: O reset da memória é CRÍTICO para garantir que cada teste é independente.
    conversational_memory.clear()

    # O agente executor imprimirá os passos devido a 'verbose=True'
    response = agent_executor.invoke({"input": q})
    pred = response["output"]
    # --------------------------------

    print(f"Agent Prediction: {pred}")

    row = {
        "question": q,
        "prediction": pred,
        "ground_truth": gt,
    }

    # --- Compute metrics ---
    # 1. Correctness (LLM-as-a-Judge)
    print("-> Computing correctness (LLM-Judge)...")
    correctness_llm_output = correctness_judge(
        inputs={"question": q},
        outputs={"prediction": pred},
        reference_outputs={"ground_truth": gt}
    )
    correctness_score = correctness_llm_output.get("score")

    # 2. Relevance (Lexical Question/Answer Overlap)
    relevance_score = relevance_evaluator(
        inputs={"question": q},
        outputs={"prediction": pred},
        reference_outputs={"ground_truth": gt}
    ).get("score")

    # 3. Similarity (Lexical Answer/Ground-Truth Match)
    similarity_score = similarity_evaluator(
        inputs={"question": q},
        outputs={"prediction": pred},
        reference_outputs={"ground_truth": gt}
    ).get("score")

    # Store results
    row["correctness_score"] = correctness_score
    row["relevance_score"] = relevance_score
    row["similarity_score"] = similarity_score
    # Guarda o feedback do LLM-as-a-Judge para debugging
    row["correctness_feedback"] = correctness_llm_output.get("feedback")

    results.append(row)

# Convert to dataframe for viewing
df = pd.DataFrame(results)
print("\n--- Evaluation Results DataFrame ---")
print(df[["question", "prediction", "correctness_score", "relevance_score", "similarity_score"]])


print("\n✅ Evaluation Loop complete.")


Starting evaluation loop...

--- Running Test Case 1/3 ---
Question: What setting should I check in a classic game to fix a stretched resolution issue?


> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: How to fix stretched resolution in classic games


Observation: 
Thought:

Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: How to fix stretched resolution in classic games
Observation: 
Thought:

 Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: How to fix stretched resolution in classic games
Observation: 
Thought:

...

Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Latest best PCSX2 settings for 'Jak 2 Renegade' in 2025
Observation: 
Thought:Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Latest best PCSX2 settings for 'Jak 2 Renegade' in 2025
Observation: 
Thought:

Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Latest best PCSX2 settings for 'Jak 2 Renegade' in 2025
Observation: 
Thought:

Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Latest best PCSX2 settings for 'Jak 2 Renegade' in 2025
Observation: 
Thought:

Do I need to use a tool? Yes
Action: retrogaming_knowledge_base
Action Input: Latest best PCSX2 settings for 'Jak 2 Renegade' in 2025
Observation: 
Thought:

> Finished chain.
Agent Prediction: Agent stopped due to iteration limit or time limit.
-> Computing correctness (LLM-Judge)...



--- Evaluation Results DataFrame ---
                                            question  \
0  What setting should I check in a classic game ...   
1  I choose PC/Mac, but I don't know my computer ...   
2  What are the latest best PCSX2 settings for 'J...   

                                          prediction  correctness_score  \
0  To fix a stretched resolution issue in a class...              False   
1  To check your computer specs on a Mac, you can...              False   
2  Agent stopped due to iteration limit or time l...              False   

   relevance_score  similarity_score  
0         0.153846          0.176471  
1         0.020833          0.197761  
2         0.000000          0.336538  

✅ Evaluation Loop complete.


##### **04 - Upload Results**

In [6]:
# ================================================================
# 04 — UPLOAD RESULTS TO LANGSMITH (Each as Its Own Run)
# ================================================================

print("\nStarting upload to LangSmith...")

client = Client(api_key=os.environ.get("LANGCHAIN_API_KEY"))
project_name = os.environ.get("LANGCHAIN_PROJECT")
now = datetime.now(timezone.utc)

try:
    for row in results:
        run_id = str(uuid.uuid4())

        # 1. Cria a Run
        client.create_run(
            id=run_id,
            project_name=project_name,
            name="HybridAgent_Evaluation_Run",
            run_type="chain",
            start_time=now,
            end_time=now,
            status="success",
            inputs={"question": row["question"]},
            outputs={"prediction": row["prediction"], "ground_truth": row["ground_truth"]},
        )

        # 2. Adiciona as métricas de avaliação como Feedback
        client.create_feedback(
            run_id=run_id,
            key="correctness",
            score=row["correctness_score"],
            comment=row["correctness_feedback"]
        )

        client.create_feedback(run_id=run_id, key="relevance", score=row["relevance_score"])
        client.create_feedback(run_id=run_id, key="similarity", score=row["similarity_score"])

    print(f"\n✅ Evaluation results uploaded to LangSmith under project: {project_name}")
    print("Access your runs at: https://smith.langchain.com")

except Exception as e:
    print(f"\n❌ FAILED to upload to LangSmith. Check your API key and permissions.")
    print(f"Error: {e}")

print("\nNotebook 05 complete. Evaluation and upload process finished.")


Starting upload to LangSmith...

❌ FAILED to upload to LangSmith. Check your API key and permissions.
Error: Failed to POST https://api.smith.langchain.com/runs in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs', '{"error":"Forbidden"}\n')

Notebook 05 complete. Evaluation and upload process finished.
